In [1]:
# 1. Clone PiCO Github Repo files onto Colab Session

!git clone https://github.com/PKU-YuanGroup/Peer-review-in-LLMs.git
%cd Peer-review-in-LLMs

Cloning into 'Peer-review-in-LLMs'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 87 (delta 10), reused 76 (delta 8), pack-reused 0
Receiving objects: 100% (87/87), 3.08 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/Peer-review-in-LLMs


In [2]:
# 2. Install requirements.txt, especially for the '"vllm" thing.
!pip install --upgrade pip
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of vllm to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of vllm to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.0 MB/s eta 0:00:00
   ━

In [3]:
''' 3. Data 1:
Upload your zipped mt_bench2, containing 'questions.jsonl' (adversarial) and 'questions3' (old)
as set up before like in our local folder. Next we'll actually get 'questions.jsonl' where it has to go.
'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!cp '/content/drive/My Drive/question.jsonl' data/

In [7]:
# MODEL ANSWER GENERATION
# FOR EACH CLOSED-SOURCE PROVIDER, SET API KEY SECRET.

import os

os.environ['OPENAI_API_KEY'] = ''

In [9]:
# Model Answer Generation (Proprietary)

# List of models for answer generation
closed_models = [
    {'model_path': 'openai/gpt-3.5-turbo', 'model_id': 'gpt-3.5-turbo'}
]

# Loop through each model and generate responses
for model in closed_models:
    !python llm_judge/gen_model_answer.py \
        --model-path {model['model_path']} \
        --model-id {model['model_id']} \
        --bench-name mt_bench

Output to data/mt_bench/model_answer/gpt-3.5-turbo.jsonl
100% 24/24 [01:41<00:00,  4.22s/it]


In [11]:
# Model Answer Generation (Open)

open_models = [
    {'model_path': 'lmsys/vicuna-7b-v1.5', 'model_id': 'vicuna-7b-v1.5'},
    {'model_path': 'microsoft/Phi-3-mini-4k-instruct', 'model_id': 'Phi-3-mini-4k-instruct'},
    {'model_path': 'lmsys/fastchat-t5-3b-v1.0', 'model_id': 'fastchat-t5-3b-v1.0'},
    {'model_path': 'THUDM/chatglm-6b', 'model_id': 'chatglm-6b'}
]

for model in open_models:
    !python llm_judge/gen_model_answer.py \
        --model-path {model['model_path']} \
        --model-id {model['model_id']} \
        --bench-name mt_bench

Output to data/mt_bench/model_answer/vicuna-7b-v1.5.jsonl
tokenizer_config.json: 100% 749/749 [00:00<00:00, 5.87MB/s]
tokenizer.model: 100% 500k/500k [00:00<00:00, 18.6MB/s]
special_tokens_map.json: 100% 438/438 [00:00<00:00, 3.95MB/s]
config.json: 100% 615/615 [00:00<00:00, 6.14MB/s]
pytorch_model.bin.index.json: 100% 26.8k/26.8k [00:00<00:00, 106MB/s]
pytorch_model-00001-of-00002.bin:   0% 0.00/9.98G [00:00<?, ?B/s]
pytorch_model-00001-of-00002.bin:   0% 10.5M/9.98G [00:00<03:35, 46.2MB/s]
pytorch_model-00001-of-00002.bin:   0% 31.5M/9.98G [00:00<01:44, 95.3MB/s]
pytorch_model-00001-of-00002.bin:   1% 73.4M/9.98G [00:00<00:51, 194MB/s] 
pytorch_model-00001-of-00002.bin:   1% 126M/9.98G [00:00<00:34, 285MB/s] 
pytorch_model-00001-of-00002.bin:   2% 178M/9.98G [00:00<00:29, 333MB/s]
pytorch_model-00001-of-00002.bin:   2% 220M/9.98G [00:00<00:28, 340MB/s]
pytorch_model-00001-of-00002.bin:   3% 262M/9.98G [00:00<00:27, 357MB/s]
pytorch_model-00001-of-00002.bin:   3% 315M/9.98G [00:01<00:

In [13]:
# After answers generated

!python llm_judge/assign_judge.py

Processing completed.
Transformation and Saving completed.
Processing completed.
Transformation and Saving completed.
Processing completed.
Transformation and Saving completed.


In [16]:
!mv llm_judge/data/judge_prompts.jsonl data/

assets	con_optimization  data	llm_judge  main.py  README.md  requirements.txt
judge_prompts.jsonl  mt_bench


In [27]:
# Judgment of Pairs; Specify (real, existing on HuggingFace) Judge Model Name or Names.
# Error: judge model smth smth
# Error: no judge_prompts file in Data
# Error:

!export OPENAI_API_KEY=''
!python llm_judge/gen_judgment.py \
--mode pairwise-all \
--new-judge-model gpt-3.5-turbo \
--model-list fastchat-t5-3b-v1.0 vicuna-7b-v1.5 gpt-3.5-turbo \
--batch-size 512 \
--bench-name mt_bench


Loaded 0 existing matches.
Successfully skipped 0 existing matches.
Loaded 0 existing matches.
Successfully skipped 0 existing matches.
Loaded 0 existing matches.
Successfully skipped 0 existing matches.
Loaded 0 existing matches.
Successfully skipped 0 existing matches.
Stats:
{
    "bench_name": "mt_bench",
    "mode": "pairwise-all",
    "judge": "gpt-3.5-turbo",
    "judge_model_path": null,
    "baseline": null,
    "model_list": [
        "fastchat-t5-3b-v1.0",
        "vicuna-7b-v1.5",
        "gpt-3.5-turbo"
    ],
    "num_gpus": 1,
    "max_gpu_memory": "35GB",
    "batch_size": 512,
    "total_num_questions": 72,
    "total_num_matches": 144,
    "output_path": "data/mt_bench/model_judgment/gpt-3.5-turbo_pair.jsonl"
}
Press Enter to confirm...
  0% 0/1 [00:00<?, ?it/s]Error formatting multi-turn prompt: 'question'
Error formatting multi-turn prompt: 'question'
Error formatting multi-turn prompt: 'question'
Error formatting multi-turn prompt: 'question'
Error formatting multi

In [32]:
# Ablation
!ls
!ls data/mt_bench/model_judgment
!cd con_optimization && python main_ablation.py --baseline 0 --mode Order --epoch 30

assets	con_optimization  data	llm_judge  main.py  README.md  requirements.txt
gpt-3.5-turbo_pair.jsonl  gpt-3.5-turbo_pair_short.jsonl
0
gpt-3.5-turbo
guanaco-33b-merged
Traceback (most recent call last):
  File "/content/Peer-review-in-LLMs/con_optimization/main_ablation.py", line 356, in <module>
    df = pd.read_json(f'../llm_judge/data/mt_bench/model_judgment/{model_str}_pair.jsonl', lines=True)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py", line 784, in read_json
    return json_reader.read()
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py", line 973, in read
    obj = self._get_object_parser(self._combine_lines(data_lines))
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py", line 1001, in _get_object_parser
    obj = FrameParser(json, **kwargs).parse()
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py", line 1134, in parse
    self._parse()
  File "/usr/local/lib/python3.10/dist-package

In [29]:
!zip -r /content/final_project_int.zip /content/Peer-review-in-LLMs

  adding: content/Peer-review-in-LLMs/ (stored 0%)
  adding: content/Peer-review-in-LLMs/requirements.txt (deflated 27%)
  adding: content/Peer-review-in-LLMs/README.md (deflated 61%)
  adding: content/Peer-review-in-LLMs/con_optimization/ (stored 0%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/ (stored 0%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.4_Peer_review_seed4.log (deflated 62%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.7_Order_seed4.log (deflated 64%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.7_Uniform_seed4.log (deflated 64%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.1_Uniform_seed4.log (deflated 64%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.1_Order_seed4.log (deflated 64%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.4_Reversed_seed4.log (deflated 78%)
  adding: content/Peer-review-in-LLMs/con_optimization/log/frac0.1_Peer_rev

In [ ]:
from google.colab import files
files.download('/content/final_project.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>